In [1]:
! ls

Data		     README.md
Data Cleaning.ipynb  SpringBoard Capstone Project 1 Proposal.pdf


In [2]:
! cd Data/

In [3]:
!ls

Data			README.md
Data Cleaning.ipynb	SpringBoard Capstone Project 1 Proposal.pdf
Datacleaning_steps.txt


In [4]:
! cd Data
!ls

Data			README.md
Data Cleaning.ipynb	SpringBoard Capstone Project 1 Proposal.pdf
Datacleaning_steps.txt


In [5]:
! ls /Data

ls: cannot access '/Data': No such file or directory


In [6]:
! ls Data

1519759656_39328483.csv  2017 PMP Data Final No Duplicates.xlsx


In [14]:
import pandas as pd

#Assign filenames
fullfile = 'Data/NM_2016_17export.csv'
ndfile = 'Data/2017 PMP Data Final No Duplicates.xlsx'

#import dataframes
fulldf = pd.read_csv(fullfile)
ND17df = pd.read_excel(ndfile)